In [2]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)
# sklearn ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"
# 공통 모듈 임포트
import numpy as np
import pandas as pd
import seaborn as sns
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)
# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "regression"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

import platform
from matplotlib import font_manager, rc
#매킨토시의 경우
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
#윈도우의 경우
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
    mpl.rcParams['axes.unicode_minus'] = False
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
        plt.savefig(path, format=fig_extension, dpi=resolution)

## 위스콘 신 유방암 데이터를 가지고 개별 훈련기와 stacking 의 차이

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [4]:
#데이터 생성
cancer_data = load_breast_cancer()

X_data = cancer_data.data
y_label = cancer_data.target

X_train , X_test , y_train , y_test = train_test_split(X_data , y_label , test_size=0.2 , random_state=42)


In [6]:
#개별 모델 생성
knn_clf = KNeighborsClassifier(n_neighbors=4)
dt_clf = DecisionTreeClassifier()
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
ada_clf = AdaBoostClassifier(n_estimators=100)

#최종 모델 생성
lr_final = LogisticRegression(C=10)

In [7]:
#개별 모델 훈련
#마지막 것만 출력됨
knn_clf.fit(X_train, y_train)
dt_clf.fit(X_train , y_train)
rf_clf.fit(X_train , y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100)

In [9]:
#개별 훈련기의 정확도 층정
knn_pred = knn_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

print('KNN 정확도:', accuracy_score(y_test, knn_pred))
print('Decision Tree의 정확도: {0:.4f}'.format(accuracy_score(y_test, dt_pred)))
print('Random Forest 정확도: {0:.4f}'.format(accuracy_score(y_test, rf_pred)))
print('Ada Boost의 정확도: {0:.4f} :'.format(accuracy_score(y_test, ada_pred)))

KNN 정확도: 0.9385964912280702
Decision Tree의 정확도: 0.9474
Random Forest 정확도: 0.9649
Ada Boost의 정확도: 0.9737 :


In [11]:
#모델이 예측한 결과를 가지고 새로운 데이터를 생성
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(pred.shape)

#행열을 전치
pred = np.transpose(pred)
print(pred.shape)

(4, 114)
(114, 4)


In [14]:
#결과를 가지고 다시 훈련
lr_final.fit(pred, y_test)
final_pred =  lr_final.predict(pred)

print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_test , final_pred)))


최종 메타 모델의 예측 정확도: 0.9737
